In [63]:
test="""Blueprint 1. Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2. Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""
source = test

In [64]:
source = open("./day_19.txt").read()

In [65]:
import numpy
import math

indicies = [1, 6, 12, 18, 21, 27, 30]
def keywords(line):
    return [math.floor(float(line[i].replace(":", ""))) for i in indicies]
blueprints = numpy.array([keywords(line.split(" ")) for line in source.splitlines()])
BLUEPRINT_INDEX=0
ORE_ROBOT_INDEX=1
CLAY_ROBOT_INDEX=2
OBSIDIAN_ROBOT_ORE_INDEX=3
OBSIDIAN_ROBOT_CLAY_INDEX=4
GEODE_ROBOT_ORE_INDEX=5
GEODE_ROBOT_OBSIDIAN_INDEX=6

blueprints[:, CLAY_ROBOT_INDEX]

ORE_INDEX=0
CLAY_INDEX=1
OBSIDIAN_INDEX=2
GEODE_INDEX=3
ORE_ROBOTS_INDEX=4
CLAY_ROBOTS_INDEX=5
OBSIDIAN_ROBOTS_INDEX=6
GEODE_ROBOTS_INDEX=7

# blueprints

In [66]:
def can_build_ore_robot(path, blueprint):
    ore = path[ORE_INDEX] - blueprint[ORE_ROBOT_INDEX]
    return ore >= 0
def build_ore_robot(path, blueprint):
    output = path.copy()
    output[ORE_INDEX] -= blueprint[ORE_ROBOT_INDEX]
    output[ORE_ROBOTS_INDEX] += 1
    return output
def can_build_clay_robot(path, blueprint):
    ore = path[ORE_INDEX]
    ore -= blueprint[CLAY_ROBOT_INDEX]
    return ore >= 0
def build_clay_robot(path, blueprint):
    output = path.copy()
    output[ORE_INDEX] -= blueprint[CLAY_ROBOT_INDEX]
    output[CLAY_ROBOTS_INDEX] += 1
    return output
def can_build_obsidian_robot(path, blueprint):
    ore = path[ORE_INDEX]
    ore -= blueprint[OBSIDIAN_ROBOT_ORE_INDEX]
    clay = path[CLAY_INDEX] - blueprint[OBSIDIAN_ROBOT_CLAY_INDEX]
    return ore >= 0 and clay >= 0
def build_obsidian_robot(path, blueprint):
    output = path.copy()
    output[ORE_INDEX] -= blueprint[OBSIDIAN_ROBOT_ORE_INDEX]
    output[CLAY_INDEX] -= blueprint[OBSIDIAN_ROBOT_CLAY_INDEX]
    output[OBSIDIAN_ROBOTS_INDEX] += 1
    return output
def can_build_geode_robot(path, blueprint):
    ore = path[ORE_INDEX] - blueprint[GEODE_ROBOT_ORE_INDEX]
    obsidian = path[OBSIDIAN_INDEX] - blueprint[GEODE_ROBOT_OBSIDIAN_INDEX]
    return ore >= 0 and obsidian >= 0
def build_geode_robot(path, blueprint):
    output = path.copy()
    output[ORE_INDEX] -= blueprint[GEODE_ROBOT_ORE_INDEX]
    output[OBSIDIAN_INDEX] -= blueprint[GEODE_ROBOT_OBSIDIAN_INDEX]
    output[GEODE_ROBOTS_INDEX] += 1
    return output
def geode_robot_distance(path, blueprint):
    ore = (blueprint[GEODE_ROBOT_ORE_INDEX] - path[ORE_INDEX])/path[ORE_INDEX] if path[ORE_INDEX]> 0 else blueprint[GEODE_ROBOT_ORE_INDEX]
    obsidian = (blueprint[GEODE_ROBOT_OBSIDIAN_INDEX]-path[OBSIDIAN_INDEX]) / path[OBSIDIAN_INDEX] if path[OBSIDIAN_INDEX]> 0 else blueprint[GEODE_ROBOT_OBSIDIAN_INDEX]
    return max(ore, obsidian)
def simulate_blueprint(blueprint, steps): 
    # print("Starting blueprint", blueprint)
    savedPaths = {}
    paths = [[0,0,0,0,1,0,0,0]]
    for step in range(steps, 0, -1):
        output = []
        # print("paths", paths)
        maxGeodes = max([path[GEODE_INDEX] + path[GEODE_ROBOTS_INDEX] for path in paths])
        if(len(paths) > 10000): 
            paths.sort(key=lambda x: x[GEODE_INDEX], reverse=True)
            paths = paths[:100000]
        for path in [path for path in paths if savedPaths.get(str(path))==None]:
            hash = str(path)
            # # print("hash is", hash)
            if(savedPaths.get(hash)!=None):
                # print("skippy")
                continue
            # if(path[GEODE_INDEX] < maxGeodes): continue
            savedPaths[hash] = path[GEODE_INDEX]
            # print("Sarting with path", path, ORE_INDEX, ORE_ROBOTS_INDEX)
            should_bother = True #geode_robot_distance(path, blueprint) < step
            should_build_ore_robot = should_bother and path[ORE_ROBOTS_INDEX] < max(blueprint[CLAY_ROBOT_INDEX], blueprint[OBSIDIAN_ROBOT_ORE_INDEX], blueprint[GEODE_ROBOT_ORE_INDEX], blueprint[ORE_ROBOT_INDEX]) and can_build_ore_robot(path, blueprint)
            should_build_clay_robot = should_bother and path[CLAY_ROBOTS_INDEX] < blueprint[OBSIDIAN_ROBOT_CLAY_INDEX] and can_build_clay_robot(path, blueprint)
            should_build_obsidian_robot = should_bother  and path[OBSIDIAN_ROBOTS_INDEX] < blueprint[GEODE_ROBOT_OBSIDIAN_INDEX] and can_build_obsidian_robot(path, blueprint)
            should_build_geode_robot = should_bother and can_build_geode_robot(path, blueprint)
            path[ORE_INDEX] += path[ORE_ROBOTS_INDEX]
            # print("ORE", path[ORE_INDEX], path[ORE_ROBOTS_INDEX])
            path[CLAY_INDEX] += path[CLAY_ROBOTS_INDEX]
            path[OBSIDIAN_INDEX] += path[OBSIDIAN_ROBOTS_INDEX]
            path[GEODE_INDEX] += path[GEODE_ROBOTS_INDEX]
            if(should_build_ore_robot):
                output.append(build_ore_robot(path,blueprint))
            if(should_build_clay_robot):
                output.append(build_clay_robot(path, blueprint))
            if(should_build_obsidian_robot):
                output.append(build_obsidian_robot(path, blueprint))
            if(should_build_geode_robot):
                # print("Building on turn ", step, "with path", path, "and blueprint", blueprint)
                output.append(build_geode_robot(path, blueprint))
            output.append(path.copy())
        # print(step, len(paths),max(numpy.array(paths)[:,GEODE_INDEX]))
        paths = output
    # print(step, len(paths),max(numpy.array(paths)[:,GEODE_INDEX]))
    return paths


In [67]:
# [blueprint[BLUEPRINT_INDEX] * max(numpy.array(simulate_blueprint(blueprint, 24))[:,3]) for blueprint in blueprints]

In [68]:
doRunPart1 = True
if doRunPart1:
    part1Score = sum([blueprint[BLUEPRINT_INDEX] * max(numpy.array(simulate_blueprint(blueprint, 24))[:,3]) for blueprint in blueprints])
    print("Part 1 score is", part1Score)

Part 1 score is 1653


In [69]:
# o = simulate_blueprint(blueprints[0], 32)

In [70]:
# print(blueprints[0:3])

doRunPart2 = True
if doRunPart2:
    part2Score = numpy.prod([max(numpy.array(simulate_blueprint(blueprint, 32))[:,3]) for blueprint in blueprints[0:3]])
    print("Part 2 score is", numpy.prod(part2Score))

Part 2 score is 4212
